In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install japanize_matplotlib  # Colab環境にjapanize_matplotlibをインストール
# その他のライブラリは、Colabではデフォルトで利用可能

import pandas as pd               # データを表のように扱うライブラリ
import numpy as np                # 数値計算を速くするライブラリ
import seaborn as sns             # きれいなグラフを簡単に作るライブラリ
import matplotlib.pyplot as plt   # グラフを作る基本的なライブラリ
%matplotlib inline
import japanize_matplotlib        # グラフに日本語を表示するライブラリ
japanize_matplotlib.japanize()

from sklearn.model_selection import train_test_split  # データを学習用とテスト用に分ける
from sklearn.metrics import roc_auc_score  # 評価の計算を行うライブラリ
from sklearn.preprocessing import StandardScaler  # 標準化の計算を行うライブラリ
import lightgbm as lgb

import warnings
warnings.simplefilter('ignore')  # 不要な警告を表示しない

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=fc440b3e976e3e01049c961a559fee81f5075ebdb784dd798e22d6f161af295e
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
import pandas as pd
from scipy.signal import butter, filtfilt

In [39]:
# ファイルを読み込む
file_path = '/content/drive/MyDrive/Colab Notebooks/修士研究/★右腕/20240423田村正規化前/★A並列データ収集CSV_0423180245_演算.csv'

In [40]:
df = pd.read_csv(file_path, encoding='latin-1')  # or try 'iso-8859-1' or 'cp1252'

In [41]:
df.head(18)

,»i¼,CXØdZTi£®j,CXØdZTi£®j.1,CXØdZTi£®j.2,Unnamed: 4
0,vªID,1,2,3,NaN
1,vªJn,04/23 18:02:45,04/23 18:02:45,04/23 18:02:45,NaN
2,TvOüg,200,200,200,NaN
3,V[PX,10097,10097,10096,NaN
4,PGAQC,x12,x12,x12,NaN
5,â³lL[,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,NaN
6,Rg,NaN,NaN,NaN,NaN
7,t@CÛ¶`®,Zf[^,Zf[^,Zf[^,NaN
8,V[PXÔ,EMG01[V],EMG01[V],EMG01[V],NaN
9,1,0,0,0,NaN


In [42]:
print(df.columns[1:4])

Index(['CXØdZTi£®j', 'CXØdZTi£®j.1',
       'CXØdZTi£®j.2'],
      dtype='object')


In [43]:
# サンプリング周波数とフィルタの設定
fs = 200  # サンプリング周波数 (Hz)
cutoff = 0.05  # カットオフ周波数 (Hz)
order = 2  # フィルタの次数

In [44]:
# 正規化カットオフ周波数
nyq = fs / 2  # ナイキスト周波数
normal_cutoff = cutoff / nyq

In [45]:
# Import the necessary module
import scipy.signal as signal

In [46]:
# ハイパスバターワースフィルターの設計
b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)

In [47]:
# 第11行目以降のデータを抽出（0-indexなので10行目以降）
filtered_data = df.iloc[10:].copy()

In [52]:
# 第2～4列目にフィルタを適用
for col in filtered_data.columns[1:4]:
    # Convert the column to numeric, handling errors by coercing non-numeric values to NaN
    filtered_data[col] = pd.to_numeric(filtered_data[col], errors='coerce')

    # Apply the filter to the numeric data
    filtered_data[col] = signal.filtfilt(b, a, filtered_data[col])

In [53]:
print(filtered_data.columns)

Index(['»i¼', 'CXØdZTi£®j',
       'CXØdZTi£®j.1', 'CXØdZTi£®j.2',
       'Unnamed: 4'],
      dtype='object')


In [54]:
print(filtered_data[col].dtype)  # データ型を確認

float64


In [55]:
# 正規化とフィルタ適用後のデータを確認
filtered_data.head()

,»i¼,CXØdZTi£®j,CXØdZTi£®j.1,CXØdZTi£®j.2,Unnamed: 4
10,2,0.003907,0.012836,NaN,NaN
11,3,-0.059852,0.014754,NaN,NaN
12,4,-0.042764,0.002861,NaN,NaN
13,5,0.030872,0.029200,NaN,NaN
14,6,0.005149,0.035279,NaN,NaN


In [18]:
# CSVファイルとして保存
filtered_data.to_csv('0423Aafter.csv', index=False)

In [19]:
# prompt: csvファイルをダウンロードしたい

from google.colab import files
files.download('0423Aafter.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>